# Imports

In [39]:
import pandas as pd
import numpy as np
from recommender import load_data

# Loading & Inspecting Data

In [40]:
# load
df = load_data()

# get first 5
pd.set_option('display.max_columns', None)
print("df.head()=================================================")
print(df.head())

# get data structure
print("df.dtypes=================================================")
print(df.dtypes)

# get num of nulls
print("df.isnul().sum()=================================================")
print(df.isnull().sum())

# get num of uniques
print("df.nunique()=================================================")
print(df.nunique())

[INFO] CSV already exists at c:\Users\wange\CS506_FinalProject\new-game-plus-backend\newgameplus\data\games.csv
df.head()=================================================
    appid                             name release_date  required_age  price  \
0     730                 Counter-Strike 2   2012-08-21             0   0.00   
1  578080              PUBG: BATTLEGROUNDS   2017-12-21             0   0.00   
2     570                           Dota 2   2013-07-09             0   0.00   
3  271590        Grand Theft Auto V Legacy   2015-04-13            17   0.00   
4  488824  Tom Clancy's Rainbow Six® Siege   2015-12-01            17  19.99   

   dlc_count                               detailed_description  \
0          1  For over two decades, Counter-Strike has offer...   
1          0  LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...   
2          2  The most-played game on Steam. Every day, mill...   
3          0  When a young street hustler, a retired bank ro...   
4          9 

# Preprocessing

In [41]:
# remove the null name rows
df = df.drop(df.loc[df['name'].isna()].index)

# drop USELESS columns
df = df.drop(['name','tags','reviews', 'appid', 'detailed_description', 'about_the_game', 'short_description', 'header_image', 'website', 'support_url','support_email','metacritic_url','notes', 'packages', 'developers', 'publishers','screenshots', 'movies','user_score','score_rank','estimated_owners','positive','negative'], axis=1) # edw: undrop reviews and sentiment thing

In [42]:
# clean data
df['release_date_cleaned'] = pd.to_datetime(df['release_date'], errors='coerce')
df = df.drop(['release_date'], axis=1)




In [43]:
print(df.head)

<bound method NDFrame.head of        required_age  price  dlc_count  windows    mac  linux  \
0                 0   0.00          1     True  False   True   
1                 0   0.00          0     True  False  False   
2                 0   0.00          2     True   True   True   
3                17   0.00          0     True  False  False   
4                17  19.99          9     True  False  False   
...             ...    ...        ...      ...    ...    ...   
94943             0   3.99          0     True  False  False   
94944             0  10.00          0     True  False  False   
94945             0   1.99          0     True  False  False   
94946             0  29.99          0     True  False  False   
94947             0   9.99          0     True  False  False   

       metacritic_score  achievements  recommendations  \
0                     0             1          4401572   
1                     0            37          1732007   
2                    90    

In [44]:
#Idetnfying the unique supported_languages
import ast 
import re

#print(type(df['supported_languages'].iloc[0]))

def clean_language(lang):
    # Remove leading/trailing whitespace and newlines
    lang = lang.strip()
    # Remove HTML-like tags (like [b][/b])
    lang = re.sub(r'\[/?b\]', '', lang)
    # Strip again after removing tags
    lang = lang.strip()
    return lang


df['supported_languages'] = df['supported_languages'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x) 

unique_supported_languages = set()

for langs in df['supported_languages']:  # Replace with your actual column name
    if isinstance(langs, list):
        for lang in langs:
            # Some entries may still have multiple langs joined with commas
            for l in lang.split(','):
                cleaned = clean_language(l)
                unique_supported_languages.add(cleaned)

unique_supported_languages = list(unique_supported_languages)
print("Unique supported languages", unique_supported_languages)


Unique supported languages ['Swahili', 'Khmer', 'Serbian', 'Ukrainian', 'Wolof', 'Kazakh', 'Icelandic', 'Norwegian', 'Malayalam', 'Tamil', 'German', 'Russian', 'Maltese', 'Zulu', 'Xhosa', 'Traditional Chinese', 'Croatian', 'Bulgarian', 'Filipino', 'Indonesian', 'Korean', 'Amharic', 'Bangla', 'Arabic', 'Konkani', 'Slovenian', 'Basque', 'Luxembourgish', 'English', 'Cherokee', 'Japanese', 'Assamese', 'Irish', 'Tigrinya', 'Kyrgyz', 'Scots', 'Hausa', 'Hungarian', 'Uzbek', 'Romanian', 'Armenian', 'Dari', 'Turkmen', 'Tswana', 'Swedish', 'Urdu', 'Yoruba', 'French', 'Hindi', 'Dutch', 'Danish', 'Macedonian', 'Polish', 'Galician', 'Estonian', 'Odia', 'Tajik', 'Latvian', 'Lithuanian', 'Georgian', 'Spanish - Spain', 'Sinhala', 'Thai', 'Nepali', 'Welsh', 'Quechua', 'Simplified Chinese', 'Finnish', 'Marathi', 'Hebrew', 'Maori', 'Belarusian', 'Persian', 'Igbo', 'Slovak', 'Malay', "K'iche'", 'Turkish', 'Spanish - Latin America', 'Tatar', 'Uyghur', 'Portuguese - Brazil', 'Gujarati', 'Kannada', 'Sotho', 

In [45]:
#One hot encoding the languages

lang_columns = {}

for lang in unique_supported_languages:
    lang_columns[lang+"_supported_languages"] = df['supported_languages'].apply(lambda x: 1 if lang in x else 0)

lang_df = pd.DataFrame(lang_columns)

df = pd.concat([df, lang_df], axis=1)

df.drop("supported_languages", axis = 1, inplace = True)

print(df.head())

   required_age  price  dlc_count  windows    mac  linux  metacritic_score  \
0             0   0.00          1     True  False   True                 0   
1             0   0.00          0     True  False  False                 0   
2             0   0.00          2     True   True   True                90   
3            17   0.00          0     True  False  False                96   
4            17  19.99          9     True  False  False                 0   

   achievements  recommendations  \
0             1          4401572   
1            37          1732007   
2             0            14337   
3            77          1803063   
4             0          1160724   

                                full_audio_languages  \
0                          ['English', 'Indonesian']   
1                                                 []   
2  ['English', 'Korean', 'Simplified Chinese', 'V...   
3             ['English', 'Spanish - Latin America']   
4  ['English', 'French', 'Italian'

In [46]:
#identify the unique supported audios
df['full_audio_languages'] = df['full_audio_languages'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x) 

unique_full_audio_languages = set()

for langs in df['full_audio_languages']:  # Replace with your actual column name
    if isinstance(langs, list):
        for lang in langs:
            # Some entries may still have multiple langs joined with commas
            for l in lang.split(','):
                cleaned = clean_language(l)
                unique_full_audio_languages.add(cleaned)

unique_full_audio_languages = list(unique_full_audio_languages)
#print("Unique Full Audio Languages:", unique_full_audio_languages)

In [47]:
#one hot encoding full audio languages

audio_lang_columns = {}

for audio_lang in unique_full_audio_languages:
    audio_lang_columns[audio_lang+"_full_audio_languages"] = df['full_audio_languages'].apply(lambda x: 1 if lang in x else 0)

audio_lang_df = pd.DataFrame(audio_lang_columns)

df = pd.concat([df, audio_lang_df], axis=1)

df.drop("full_audio_languages", axis=1, inplace=True)


print(df.head())

   required_age  price  dlc_count  windows    mac  linux  metacritic_score  \
0             0   0.00          1     True  False   True                 0   
1             0   0.00          0     True  False  False                 0   
2             0   0.00          2     True   True   True                90   
3            17   0.00          0     True  False  False                96   
4            17  19.99          9     True  False  False                 0   

   achievements  recommendations  \
0             1          4401572   
1            37          1732007   
2             0            14337   
3            77          1803063   
4             0          1160724   

                                          categories  \
0  ['Multi-player', 'Cross-Platform Multiplayer',...   
1  ['Multi-player', 'PvP', 'Online PvP', 'Stats',...   
2  ['Multi-player', 'Co-op', 'Steam Trading Cards...   
3  ['Single-player', 'Multi-player', 'PvP', 'Onli...   
4  ['Single-player', 'Multi-player

In [50]:
#Identifying the unique genres
import ast
df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

unique_genres = set()

for genres in df['genres']:
    if isinstance(genres, list):
        unique_genres.update(genres)

unique_genre_list = list(unique_genres)
print("Unique genres:", unique_genre_list)

KeyError: 'genres'

In [51]:
#One hot encoding the genres
for genre in unique_genres:
    df[genre] = df['genres'].apply(lambda x: 1 if genre in x else 0)

df.drop("genres", axis=1, inplace=True)


print(df.head())

KeyError: 'genres'

In [52]:
#identifying unique categories

import ast
df['categories'] = df['categories'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

unique_categories = set()

for categories in df['categories']:
    if isinstance(categories, list):
        unique_categories.update(categories)

unique_categories_list = list(unique_categories)
print("Unique categories:", unique_categories_list)

Unique categories: ['In-App Purchases', 'Steam Trading Cards', 'Single-player', 'Partial Controller Support', 'LAN Co-op', 'Co-op', 'Full controller support', 'MMO', 'VR Only', 'VR Support', 'Captions available', 'Shared/Split Screen Co-op', 'Steam Leaderboards', 'Cross-Platform Multiplayer', 'Commentary available', 'Steam Workshop', 'Online PvP', 'Includes Source SDK', 'VR Supported', 'Steam Cloud', 'Remote Play on Tablet', 'SteamVR Collectibles', 'Online Co-op', 'Multi-player', 'Remote Play on Phone', 'Tracked Controller Support', 'Valve Anti-Cheat enabled', 'Steam Achievements', 'Steam Turn Notifications', 'PvP', 'Remote Play Together', 'Includes level editor', 'Stats', 'Shared/Split Screen', 'HDR available', 'Shared/Split Screen PvP', 'Family Sharing', 'LAN PvP', 'Steam Timeline', 'Remote Play on TV']


In [53]:
#One hot encoding the genres

for categories in unique_categories:
    df[categories] = df['categories'].apply(lambda x: 1 if categories in x else 0)

df.drop("categories", axis=1, inplace=True)

print(df.head())

   required_age  price  dlc_count  windows    mac  linux  metacritic_score  \
0             0   0.00          1     True  False   True                 0   
1             0   0.00          0     True  False  False                 0   
2             0   0.00          2     True   True   True                90   
3            17   0.00          0     True  False  False                96   
4            17  19.99          9     True  False  False                 0   

   achievements  recommendations  average_playtime_forever  \
0             1          4401572                     33189   
1            37          1732007                         0   
2             0            14337                     43031   
3            77          1803063                     19323   
4             0          1160724                         0   

   average_playtime_2weeks  median_playtime_forever  median_playtime_2weeks  \
0                      879                     5174                     350   
